Matching Text With Embeddings
-----

Given a short piece of text, find the closest matches from a group of documents.

In [30]:
import gensim
import gensim.downloader as api
from   gensim.utils import tokenize

In [31]:
def preprocess_text(text):
    return list(tokenize(text.lower()))

In [32]:
# Setup models
model = api.load('word2vec-google-news-300')

Define data
----

Let's match resumes to job postings.

In [40]:
job_postings = ["We are looking for an experienced and passionate teacher to join our team! The ideal candidate will have a Bachelor's degree and be certified to teach in the state of Anytown. We are looking for someone with a deep commitment to helping children learn and develop, as well as the ability to create a supportive and stimulating learning environment.",
                "We are looking for an experienced and talented cook to join our team. The ideal candidate will have a commitment to excellence and a passion for food. The cook will be responsible for preparing meals in accordance with established recipes and standards.",
                "We are seeking a highly motivated and customer service-oriented individual to join our team as a Waiter/Waitress. In this role, you will be responsible for taking orders, serving food and drinks, and providing friendly customer service to guests. The ideal candidate will possess excellent communication and customer service skills, have a positive attitude, and the ability to work in a fast-paced environment. Knowledge of food and beverage service is preferred but not required.",
                "We are looking for an experienced and highly motivated teacher. Ideally, you are a creative and enthusiastic individual who is dedicated to providing students with an exceptional learning experience.",
               ]
job_postings_processed = [preprocess_text(jp) for jp in job_postings]

In [41]:
resume = "Seeking education position" # Note that "education" does not appear in any of the job postings
resume_processed = preprocess_text(resume)

Matching
----

Let's use the Word Mover's Distance (WMD) algorithm to match text documents. 


<br>
<center><img src="images/wmd.png" width="75%"/></center>

WMD assess the “distance” between two documents in a meaningful way even when they have no words in common. WMD used the vector embeddings of words. WMD then finds the minimum “traveling distance” between the collection of word vectors. In other words the most efficient way to “move” the distribution of one document's vectors to the distribution of another documents  vector.

In [42]:
def calculate_distances(resume, job_postings, model):
    results = {}
    for i, job_post in enumerate(job_postings):
        results[i] = model.wmdistance(resume, job_post)
        
    # Sort the results by closest distance
    results = sorted(results.items(), key=lambda x: x[1])
    
    return results

In [43]:
# This can take a little time because of the WMD algorithm
results = calculate_distances(resume_processed, job_postings_processed, model)

In [44]:
# Let's look at the top match
idx_top_match = results[0][0]
job_postings[idx_top_match]

'We are looking for an experienced and highly motivated teacher. Ideally, you are a creative and enthusiastic individual who is dedicated to providing students with an exceptional learning experience.'

In [45]:
# Let's look at the second match
idx_second_match = results[1][0]
job_postings[idx_second_match]

"We are looking for an experienced and passionate teacher to join our team! The ideal candidate will have a Bachelor's degree and be certified to teach in the state of Anytown. We are looking for someone with a deep commitment to helping children learn and develop, as well as the ability to create a supportive and stimulating learning environment."

In [46]:
# Another example
resume = "I like making food" # Note this should return cook postings over server postings
resume_processed = preprocess_text(resume)
results2 = calculate_distances(resume_processed, job_postings_processed, model)
idx_top_match = results2[0][0]
job_postings[idx_top_match]

'We are looking for an experienced and talented cook to join our team. The ideal candidate will have a commitment to excellence and a passion for food. The cook will be responsible for preparing meals in accordance with established recipes and standards.'

<br>
<br>
<br>

------